In [1]:
import pandas as pd

In [2]:
train = pd.read_csv("train.csv")
test = pd.read_csv("test.csv")
ss = pd.read_csv("sample_submission.csv")

In [3]:
train.head()

,oid,category,text
0,365271984,winter_sport,Волшебные фото Виктория Поплавская ЕвгенияМедв...
1,503385563,extreme,Возвращение в подземелье Треша 33 Эйфория тупо...
2,146016084,football,Лучшие чешские вратари – Доминик Доминатор Гаш...
3,933865449,boardgames,Rtokenoid Warhammer40k валрак решил нас подкор...
4,713550145,hockey,Шестеркин затаскивает Рейнджерс в финал Восточ...


In [4]:
train.category.unique()

array(['winter_sport', 'extreme', 'football', 'boardgames', 'hockey',
       'esport', 'athletics', 'motosport', 'basketball', 'tennis',
       'autosport', 'martial_arts', 'volleyball'], dtype=object)

In [5]:
test.head()

,oid,text
0,749208109,СПОЧНО СООБЩЕСТВО ПРОДАЕТСЯ ЗА 1300Р ЗА ПОКУПК...
1,452466036,Естественное восстановление после тяжелой трен...
2,161038103,Тема нарядов продолжается Одна из британских ж...
3,663621910,Привет Избранный. Ты спрашиваешь себя ЧТО здес...
4,566255305,КОРОЛЬ ПЯТИСОТНИКОВ В ДЕЛЕ Андрей Рублев успеш...


### test and train oid intersection

In [6]:
# oid intersection: []
def intersection(lst1, lst2):
    lst3 = [value for value in lst1 if value in lst2]
    return lst3

print(intersection(train.oid.tolist(), test.oid.tolist()))

### drop text with different labels

In [7]:
# text with different labels
tmp = train[["oid", "text", "category"]].groupby("text").agg(list)
tmp["counts"] = tmp.category.apply(lambda x: len(set(x)))
tmp = tmp[tmp.counts >= 2].reset_index()
noise_text = set(tmp.text.tolist())
tmp.head()

,text,oid,category,counts
0,30 дней Okko бесплатно 33 Подключайтесь и смот...,"[410518766, 749181195]","[hockey, martial_arts]",2
1,60 дней Плюса Мульти в подарок по промокоду MY...,"[457739190, 200619851, 863906059, 910250450]","[volleyball, basketball, esport, volleyball]",3
2,60 дней Плюса Мульти в подарок по промокоду MY...,"[363941572, 609061254, 587012161, 821548675, 5...","[football, martial_arts, football, athletics, ...",4
3,Python и Java изучаем языки программирования н...,"[193884872, 805730295]","[football, martial_arts]",2
4,Админ здравствуйте. Пропустите или нет – не зн...,"[123162769, 460666149, 864379998]","[extreme, esport, extreme]",2


In [8]:
print("Count unique oid in train: ", train.oid.nunique())
# remove noise text from train
tmp_train = train[~train.text.isin(noise_text)]
print("Count noisy rows in train: ", len(train) - len(tmp_train))
print("Count unique oid in train without noise text: ", train.oid.nunique())

Count unique oid in train:  3874
Count noisy rows in train:  489
Count unique oid in train without noise text:  3874


In [9]:
print("Count unique oid in train: ", test.oid.nunique())
# remove noise text from test
tmp_test = test[~test.text.isin(noise_text)]
print("Count noisy rows in test: ", len(test) - len(tmp_test))
print("Count unique oid in train without noise text: ", test.oid.nunique())

Count unique oid in train:  2626
Count noisy rows in test:  162
Count unique oid in train without noise text:  2626


### datasets with unique texts

In [10]:
print(tmp_train.shape)
tmp_train = tmp_train.drop_duplicates(subset=['text'])
print(tmp_train.shape)

(38251, 3)
(35631, 3)


In [11]:
print(tmp_test.shape)
tmp_test = tmp_test.drop_duplicates(subset=['oid', 'text'])
print(tmp_test.shape)

(26098, 2)
(25893, 2)


In [12]:
print("Count unique oid in train without duplicates: ", tmp_train.oid.nunique())
print("Count unique oid in test without duplicates: ", tmp_test.oid.nunique())

Count unique oid in train without duplicates:  3874
Count unique oid in test without duplicates:  2626


### new target distribution

In [14]:
tmp_train.category.value_counts(normalize=True)

autosport       0.084393
tennis          0.082148
hockey          0.080969
martial_arts    0.080155
football        0.077769
volleyball      0.077685
extreme         0.076899
winter_sport    0.076450
motosport       0.076310
esport          0.075440
athletics       0.072465
boardgames      0.070557
basketball      0.068760
Name: category, dtype: float64